In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parents[1]

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("qepc in root?", (PROJECT_ROOT / "qepc").exists())


PROJECT_ROOT: C:\Users\wdors\qepc_project
qepc in root? True


In [2]:
from qepc.brain.games_loader import fetch_league_games, build_games_table
from qepc.brain.scripts import label_game_scripts_by_total_points

# 👇 switch this to "2025-26" later when you want to run current season
season = "2023-24"

team_games = fetch_league_games(season)
games_df = build_games_table(team_games)

print("Season:", season)
print("games_df rows:", len(games_df))
display(games_df.head())

scripts_df = label_game_scripts_by_total_points(
    games_df,
    low_quantile=0.25,
    high_quantile=0.75,
)

print("scripts_df shape:", scripts_df.shape)
display(scripts_df.head())

print("\nScript label distribution:")
print(scripts_df["SCRIPT_LABEL"].value_counts(normalize=True))


[games_loader] Fetching LeagueGameLog for season=2023-24, season_type=Regular Season...
[games_loader] Retrieved 2460 team-games.
Season: 2023-24
games_df rows: 1230


,GAME_ID,GAME_DATE,SEASON_ID,HOME_TEAM_ID,HOME_TEAM_NAME,HOME_TEAM_ABBREVIATION,HOME_PTS,AWAY_TEAM_ID,AWAY_TEAM_NAME,AWAY_TEAM_ABBREVIATION,AWAY_PTS,HOME_RESULT,MARGIN,TOTAL_POINTS
0,0022300061,2023-10-24,22023,1610612743,Denver Nuggets,DEN,119,1610612747,Los Angeles Lakers,LAL,107,W,12,226
1,0022300062,2023-10-24,22023,1610612744,Golden State Warriors,GSW,104,1610612756,Phoenix Suns,PHX,108,L,-4,212
2,0022300070,2023-10-25,22023,1610612741,Chicago Bulls,CHI,104,1610612760,Oklahoma City Thunder,OKC,124,L,-20,228
3,0022300074,2023-10-25,22023,1610612746,LA Clippers,LAC,123,1610612757,Portland Trail Blazers,POR,111,W,12,234
4,0022300068,2023-10-25,22023,1610612748,Miami Heat,MIA,103,1610612765,Detroit Pistons,DET,102,W,1,205


scripts_df shape: (1230, 8)


,GAME_ID,GAME_DATE,TOTAL_POINTS,TOTAL_Q,SCRIPT_LABEL,SCRIPT_INDEX,TOTAL_Q_LOW,TOTAL_Q_HIGH
0,0022300061,2023-10-24,226,0.472358,BALANCED,1,214.0,242.0
1,0022300062,2023-10-24,212,0.213821,GRIND,0,214.0,242.0
2,0022300070,2023-10-25,228,0.510976,BALANCED,1,214.0,242.0
3,0022300074,2023-10-25,234,0.623171,BALANCED,1,214.0,242.0
4,0022300068,2023-10-25,205,0.114634,GRIND,0,214.0,242.0



Script label distribution:
SCRIPT_LABEL
BALANCED    0.496748
GRIND       0.252033
CHAOS       0.251220
Name: proportion, dtype: float64


In [3]:
from qepc.brain.teams_loader import fetch_league_team_season_stats

# Base counting stats
team_stats_base = fetch_league_team_season_stats(
    season,
    measure_type="Base",
)

print("team_stats_base shape:", team_stats_base.shape)
display(team_stats_base.head())

# Advanced stats (OFF_RATING, DEF_RATING, PACE, etc.)
team_stats_adv = fetch_league_team_season_stats(
    season,
    measure_type="Advanced",
)

print("team_stats_adv shape:", team_stats_adv.shape)
display(team_stats_adv.head())


team_stats_base shape: (30, 54)


,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,1610612737,Atlanta Hawks,82,36,46,0.439,3971.0,3529,7584,0.465,...,6,16,16,16,26,22,15,7,5,21
1,1610612738,Boston Celtics,82,64,18,0.780,3966.0,3601,7396,0.487,...,2,14,1,27,1,1,2,30,2,1
2,1610612751,Brooklyn Nets,82,32,50,0.390,3961.0,3334,7307,0.456,...,11,20,12,28,12,16,13,21,25,22
3,1610612766,Charlotte Hornets,82,21,61,0.256,3946.0,3281,7133,0.460,...,30,26,18,25,25,13,7,29,28,30
4,1610612741,Chicago Bulls,82,39,43,0.476,3996.0,3448,7339,0.470,...,14,23,3,9,18,14,17,14,22,20


team_stats_adv shape: (30, 46)


,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,1610612737,Atlanta Hawks,82,36,46,0.439,3971.0,114.0,116.4,116.7,...,18,21,5,16,12,13,17,18,6,22
1,1610612738,Boston Celtics,82,64,18,0.780,3966.0,120.2,122.2,109.0,...,3,13,14,7,5,1,2,1,19,1
2,1610612751,Brooklyn Nets,82,32,50,0.390,3961.0,110.4,112.4,113.1,...,19,19,12,19,20,14,25,27,25,21
3,1610612766,Charlotte Hornets,82,21,61,0.256,3946.0,107.1,108.6,117.0,...,25,22,27,22,29,21,26,28,22,29
4,1610612741,Chicago Bulls,82,39,43,0.476,3996.0,112.4,114.0,113.2,...,13,24,11,17,14,3,22,21,28,20


In [4]:
import pandas as pd

# Keep only the columns we care about from advanced stats
adv_cols_keep = [
    "TEAM_ID",
    "TEAM_NAME",
    "TEAM_ABBREVIATION",
    "GP",
    "W",
    "L",
    "W_PCT",
    "MIN",
    "OFF_RATING",
    "DEF_RATING",
    "NET_RATING",
    "PACE",
    "PIE",
]

adv_cols_keep = [c for c in adv_cols_keep if c in team_stats_adv.columns]
team_adv_small = team_stats_adv[adv_cols_keep].copy()

print("team_adv_small columns:", list(team_adv_small.columns))
display(team_adv_small.head())

# Make home/away copies with prefixes
home_adv = team_adv_small.add_prefix("HOME_")  # TEAM_ID -> HOME_TEAM_ID
away_adv = team_adv_small.add_prefix("AWAY_")  # TEAM_ID -> AWAY_TEAM_ID

# Start from games_df
games_feat = games_df.copy()

# Merge home team stats:
games_feat = games_feat.merge(
    home_adv,
    left_on="HOME_TEAM_ID",   # from games_df
    right_on="HOME_TEAM_ID",  # from home_adv (prefixed TEAM_ID)
    how="left",
)

# Merge away team stats:
games_feat = games_feat.merge(
    away_adv,
    left_on="AWAY_TEAM_ID",   # from games_df
    right_on="AWAY_TEAM_ID",  # from away_adv (prefixed TEAM_ID)
    how="left",
)

print("games_feat shape after merges:", games_feat.shape)
display(games_feat.head())


team_adv_small columns: ['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'PACE', 'PIE']


,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,OFF_RATING,DEF_RATING,NET_RATING,PACE,PIE
0,1610612737,Atlanta Hawks,82,36,46,0.439,3971.0,116.4,118.4,-2.0,100.84,0.482
1,1610612738,Boston Celtics,82,64,18,0.780,3966.0,122.2,110.6,11.7,97.98,0.555
2,1610612751,Brooklyn Nets,82,32,50,0.390,3961.0,112.4,115.4,-2.9,97.56,0.483
3,1610612766,Charlotte Hornets,82,21,61,0.256,3946.0,108.6,119.2,-10.6,97.81,0.447
4,1610612741,Chicago Bulls,82,39,43,0.476,3996.0,114.0,115.7,-1.7,96.94,0.493


games_feat shape after merges: (1230, 36)


,GAME_ID,GAME_DATE,SEASON_ID,HOME_TEAM_ID,HOME_TEAM_NAME_x,HOME_TEAM_ABBREVIATION,HOME_PTS,AWAY_TEAM_ID,AWAY_TEAM_NAME_x,AWAY_TEAM_ABBREVIATION,...,AWAY_GP,AWAY_W,AWAY_L,AWAY_W_PCT,AWAY_MIN,AWAY_OFF_RATING,AWAY_DEF_RATING,AWAY_NET_RATING,AWAY_PACE,AWAY_PIE
0,0022300061,2023-10-24,22023,1610612743,Denver Nuggets,DEN,119,1610612747,Los Angeles Lakers,LAL,...,82,47,35,0.573,3971.0,115.4,114.8,0.6,101.38,0.518
1,0022300062,2023-10-24,22023,1610612744,Golden State Warriors,GSW,104,1610612756,Phoenix Suns,PHX,...,82,49,33,0.598,3956.0,116.8,113.7,3.1,99.00,0.527
2,0022300070,2023-10-25,22023,1610612741,Chicago Bulls,CHI,104,1610612760,Oklahoma City Thunder,OKC,...,82,57,25,0.695,3961.0,118.3,111.0,7.3,100.85,0.538
3,0022300074,2023-10-25,22023,1610612746,LA Clippers,LAC,123,1610612757,Portland Trail Blazers,POR,...,82,21,61,0.256,3976.0,107.6,116.6,-9.0,97.88,0.434
4,0022300068,2023-10-25,22023,1610612748,Miami Heat,MIA,103,1610612765,Detroit Pistons,DET,...,82,14,68,0.171,3951.0,109.0,118.0,-9.0,100.45,0.449


In [5]:
games_with_scripts = games_feat.merge(
    scripts_df[["GAME_ID", "SCRIPT_LABEL", "SCRIPT_INDEX"]],
    on="GAME_ID",
    how="left",
)

print("games_with_scripts shape:", games_with_scripts.shape)
display(games_with_scripts.head())


games_with_scripts shape: (1230, 38)


,GAME_ID,GAME_DATE,SEASON_ID,HOME_TEAM_ID,HOME_TEAM_NAME_x,HOME_TEAM_ABBREVIATION,HOME_PTS,AWAY_TEAM_ID,AWAY_TEAM_NAME_x,AWAY_TEAM_ABBREVIATION,...,AWAY_L,AWAY_W_PCT,AWAY_MIN,AWAY_OFF_RATING,AWAY_DEF_RATING,AWAY_NET_RATING,AWAY_PACE,AWAY_PIE,SCRIPT_LABEL,SCRIPT_INDEX
0,0022300061,2023-10-24,22023,1610612743,Denver Nuggets,DEN,119,1610612747,Los Angeles Lakers,LAL,...,35,0.573,3971.0,115.4,114.8,0.6,101.38,0.518,BALANCED,1
1,0022300062,2023-10-24,22023,1610612744,Golden State Warriors,GSW,104,1610612756,Phoenix Suns,PHX,...,33,0.598,3956.0,116.8,113.7,3.1,99.00,0.527,GRIND,0
2,0022300070,2023-10-25,22023,1610612741,Chicago Bulls,CHI,104,1610612760,Oklahoma City Thunder,OKC,...,25,0.695,3961.0,118.3,111.0,7.3,100.85,0.538,BALANCED,1
3,0022300074,2023-10-25,22023,1610612746,LA Clippers,LAC,123,1610612757,Portland Trail Blazers,POR,...,61,0.256,3976.0,107.6,116.6,-9.0,97.88,0.434,BALANCED,1
4,0022300068,2023-10-25,22023,1610612748,Miami Heat,MIA,103,1610612765,Detroit Pistons,DET,...,68,0.171,3951.0,109.0,118.0,-9.0,100.45,0.449,GRIND,0


In [6]:
import numpy as np

feature_cols = [
    "HOME_OFF_RATING",
    "HOME_DEF_RATING",
    "HOME_NET_RATING",
    "HOME_PACE",
    "HOME_PIE",
    "AWAY_OFF_RATING",
    "AWAY_DEF_RATING",
    "AWAY_NET_RATING",
    "AWAY_PACE",
    "AWAY_PIE",
]

feature_cols = [c for c in feature_cols if c in games_with_scripts.columns]

print("Using feature columns:", feature_cols)

# Drop rows where we don't have a script label (shouldn't be many, but just in case)
model_df = games_with_scripts.dropna(subset=["SCRIPT_INDEX"]).copy()

X = model_df[feature_cols].values.astype(float)
y = model_df["SCRIPT_INDEX"].values.astype(int)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Script label counts:", np.bincount(y))


Using feature columns: ['HOME_OFF_RATING', 'HOME_DEF_RATING', 'HOME_NET_RATING', 'HOME_PACE', 'HOME_PIE', 'AWAY_OFF_RATING', 'AWAY_DEF_RATING', 'AWAY_NET_RATING', 'AWAY_PACE', 'AWAY_PIE']
X shape: (1230, 10)
y shape: (1230,)
Script label counts: [310 611 309]


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Track original row indices so we can map predictions back to model_df rows
idx = np.arange(len(model_df))

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X,
    y,
    idx,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

clf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    n_jobs=-1,
)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("\nClassification report (SCRIPT_INDEX):")
print(classification_report(y_test, y_pred, digits=3))

print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))


Train size: 984
Test size: 246

Classification report (SCRIPT_INDEX):
              precision    recall  f1-score   support

           0      0.394     0.419     0.406        62
           1      0.536     0.549     0.543       122
           2      0.400     0.355     0.376        62

    accuracy                          0.467       246
   macro avg      0.443     0.441     0.442       246
weighted avg      0.466     0.467     0.466       246


Confusion matrix:
[[26 26 10]
 [32 67 23]
 [ 8 32 22]]


In [8]:
probs = clf.predict_proba(X_test)

print("probs shape:", probs.shape)  # (n_test, 3)

# Map class indices (0=GRIND, 1=BALANCED, 2=CHAOS) to columns in probs
class_order = list(clf.classes_)   # e.g. [0, 1, 2]
i_grind = class_order.index(0)
i_bal   = class_order.index(1)
i_chaos = class_order.index(2)

# Take the first 10 test samples
test_idx_subset = idx_test[:10]

# Columns we’d like to see for context
preview_cols = [
    "GAME_ID",
    "HOME_TEAM_NAME",
    "AWAY_TEAM_NAME",
    "TOTAL_POINTS",
    "SCRIPT_LABEL",
]

# Keep only the ones that actually exist in model_df
preview_cols = [c for c in preview_cols if c in model_df.columns]

# Grab those rows from model_df
results_preview = model_df.iloc[test_idx_subset][preview_cols].copy()

# Attach predicted probabilities
results_preview["P_GRIND"]    = probs[:10, i_grind]
results_preview["P_BALANCED"] = probs[:10, i_bal]
results_preview["P_CHAOS"]    = probs[:10, i_chaos]

display(results_preview)


probs shape: (246, 3)


,GAME_ID,TOTAL_POINTS,SCRIPT_LABEL,P_GRIND,P_BALANCED,P_CHAOS
576,0022300544,200,GRIND,0.749167,0.187556,0.063278
99,0022300153,219,BALANCED,0.721944,0.180556,0.097500
203,0022300042,230,BALANCED,0.077361,0.414782,0.507857
177,0022300027,229,BALANCED,0.742778,0.174833,0.082389
217,0022300231,202,GRIND,0.123944,0.798278,0.077778
1221,0022301200,203,GRIND,0.037778,0.800889,0.161333
829,0022300800,240,BALANCED,0.176667,0.198556,0.624778
199,0022300219,271,CHAOS,0.010000,0.818500,0.171500
37,0022300095,223,BALANCED,0.710159,0.160563,0.129278
505,0022300470,227,BALANCED,0.585667,0.266000,0.148333


In [9]:
import joblib
import pandas as pd

out_dir = PROJECT_ROOT / "data" / "processed" / "nba" / "models"
out_dir.mkdir(parents=True, exist_ok=True)

# 1) Sanity-check that clf and model_df exist
print("clf type:", type(clf))
print("model_df shape:", model_df.shape)

# 2) Save the classifier
model_path = out_dir / f"script_classifier_rf_{season}.joblib"
joblib.dump(clf, model_path)
print("Saved model to:", model_path)

# 3) Copy model_df and drop duplicate columns to keep parquet happy
model_df_out = model_df.copy()

# If there are duplicate column names (common after merges), parquet will explode.
# This keeps only the first occurrence of each column name.
model_df_out = model_df_out.loc[:, ~model_df_out.columns.duplicated()]

print("model_df_out shape after dropping duplicate columns:", model_df_out.shape)

dataset_path = out_dir / f"script_classifier_dataset_{season}.parquet"
model_df_out.to_parquet(dataset_path, index=False)
print("Saved dataset to:", dataset_path)

print("✅ Saved model + dataset to:", out_dir)


clf type: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
model_df shape: (1230, 38)
Saved model to: C:\Users\wdors\qepc_project\data\processed\nba\models\script_classifier_rf_2023-24.joblib
model_df_out shape after dropping duplicate columns: (1230, 38)
Saved dataset to: C:\Users\wdors\qepc_project\data\processed\nba\models\script_classifier_dataset_2023-24.parquet
✅ Saved model + dataset to: C:\Users\wdors\qepc_project\data\processed\nba\models
